بسم الله الرحمن الرحيم 

Credit: Adapted from https://www.tensorflow.org/text/guide/subwords_tokenizer

In [70]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_text as text
import functools
from datasets import load_dataset
from datasets import Dataset
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [63]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

In [3]:
#loads 100k-words dataset (credit: @ammar)
dataset = load_dataset("dot-ammar/AR-dotless-small")

In [5]:
#build tf.data.Dataset for clean column from HuggingFace dataset
train_clean = dataset['train'].to_tf_dataset(
    columns = 'clean',
    prefetch = False
)

In [6]:
#same for dotless column
train_dtl = dataset['train'].to_tf_dataset(
    columns = 'dotless',
    prefetch = False
)

## Generate Vocabulary

In [ ]:
#commented out so that I don't accidentally run it

"""
#generate vocabulary for 'clean' text with 50000 words/subwords

%%time
clean_vocab = bert_vocab.bert_vocab_from_dataset(
    dataset = train_clean,
    vocab_size = 50000,
    reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]
)

#function to write vocab to file
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

#produce vocab file for clean_vocab
write_vocab_file('clean_vocab.txt', clean_vocab)

#generate vocabulary for 'dotless' text with 35000 words/subwords

%%time
dotless_vocab = bert_vocab.bert_vocab_from_dataset(
    dataset = train_dtl,
    vocab_size = 35000,
    reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]
)

#produce vocab file for dotless_vocab
write_vocab_file('dotless_vocab.txt', dotless_vocab)
"""

## Tokenization

In [14]:
def build_vocab(filepath):
  vocab_list = []
  f = open(filepath, "r")
  for aLine in f:
    vocab_list.append(aLine)
  return vocab_list

In [15]:
clean_vocab = build_vocab('clean_vocab.txt')

In [32]:
clean_tokenizer = text.BertTokenizer('clean_vocab.txt')
dotless_tokenizer = text.BertTokenizer('dotless_vocab.txt')

In [ ]:
def decode_string(ints):
  strs = [chr(i) for i in ints]
  joined = [''.join(strs)]
  return joined

In [87]:
for cln in train_clean.take(3):
    # Tokenize the examples -> (batch, word, word-piece)
    token_batch = clean_tokenizer.tokenize(cln)
    # Merge the word and word-piece axes -> (batch, tokens)
    token_batch = token_batch.merge_dims(-2,-1)

    words = clean_tokenizer.detokenize(token_batch)
    aTensor = tf.strings.reduce_join(words, separator=' ', axis=-1)

    decoded = tf.strings.unicode_decode(aTensor, 'utf-8').numpy()
    decoded_list = [decode_string(ex) for ex in decoded]
    print(decoded_list)

[['في حادثة تؤكد الواقع السي للصحافة السورية والاستهتار بها والحواجز التي تصطدم بها في عملها أصدر مدير فرع الهيئة المركزية للرقابة والتفتيش في السويدا قرارا يتضمن منع دخول مراسلة الوطن في المحافظة لمقر الفرعصحيفة الوطن التي أوردت الخبر تسالت عن السر الذي يمكن أن يخفيه المدير الذي يمنع صحفي من الدخول إلى دائرة حكومية متناسيا أنها ملك عام ولايحق له اتخاذ مثل هذا القرار مشيرة إلى أن المراسلة حاولت أكثر من مرة لقا المدير وسؤاله عن سبب إصدار القرار لكنه رفض لقاهاتحليلات الصحيفة وصلت إلى أن القرار يمكن أن يعود لقيامها سابقا بالإضاة على أماكن كثيرة من الخلل في عمل التفتيش عموما ومدير فرع الهيئة خصوصا إضافة إلى تسليط الضو على التجاوزات والقرارات التعسفية بحق كثير من المشتكين ممن تقدموا بشكواهم خطيا إلى الصحيفةيشار إلى أن الصحفيين في سوريا يتعرضون للكثير من المواقف المماثلة ويعانون الكثير للحصول على المعلومات في ظل قرارات حكومية لاتمكنهم الحصول عليها بيسر وسهولة حيث سبق أن منع مدير مدرسة خاصة في دمشق مراسل صحيفة تشرين من الدخول والحصول على المعلومات اللازمة لإنجاز تقرير صحفي عن المدارس الخاصةأخ